In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json
from datetime import date
import pandas as pd
from bs4 import BeautifulSoup
import re
import string
import numpy as np
import os

In [2]:
# Establecemos las rutas de lectura y escritura de ficheros
write_path = '../data/news_test.csv'
read_path = '../data/urls.csv'

In [6]:
# Obtenemos la lista de urls scrapeadas en el día de hoy
today = np.datetime64(date.today())
df_urls = pd.read_csv(read_path)
df_urls['date'] = pd.to_datetime(df_urls['date'], format="%Y/%m/%d")
dates = df_urls['date']
keep = dates == today
df_urls = df_urls[keep]
urls = list(df_urls['url'])
df_urls.head()

,url,date
2726,https://www.20minutos.es/noticia/4375123/0/edu...,2020-09-10
2727,https://www.20minutos.es/noticia/4373963/0/ast...,2020-09-10
2728,https://www.20minutos.es/noticia/4375204/0/dav...,2020-09-10
2729,https://www.20minutos.es/noticia/4375219/0/la-...,2020-09-10
2730,https://www.20minutos.es/noticia/4362007/0/los...,2020-09-10


In [4]:
# Creamos la clase Spider
# Extrayendo el titular y el texto de la noticia, menos en los periódicos de suscripción

class news_spider(scrapy.Spider):
    name = 'blogspider'
    start_urls = urls
    
    # Inicalizamos el fichero de escritura con las cabeceras
    def __init__(self):
        if os.stat(write_path).st_size == 0:
            print(f"scraping_date,url,headline,text,lang", file=filep)

    def parse(self, response):

        list_replace = ['\n', '"', ',']
        extract_text = ""
        lang = 'es'

        if response.url.startswith('https://www.publico.es/'):
            headline = response.css('h1').get()
            # Elimina las etiquetas span y su contenido
            headline = re.sub(r'<span>.+</span>','',headline)
            extract_text = response.css('p.pb-article-item-iteration').getall()
        
        elif response.url.startswith('https://www.20minutos.es'):
            headline = response.css('main section div h1 ::text').extract_first().strip()
            extract_text = response.css('section article div p ::text').getall()

        elif response.url.startswith('https://www.larazon.es/'):
            headline = response.css('h1.headline').get()
            extract_text = response.css('p.body-components__text').getall()

        elif response.url.startswith('https://elpais.com/'):
            headline = response.css('h1.a_t ::text').get()
            extract_text = 'Suscripción'

        elif response.url.startswith('https://www.elmundo.es/'):
            headline = response.css('h1.ue-c-article__headline ::text').get()
            extract_text = 'Suscripción'

        elif response.url.startswith('https://time.com'):
            headline = response.css('h1.headline').get()
            extract_text = response.css('p').getall()
            lang = 'en'
            
        elif response.url.startswith('https://www.abc.es/'):
            headline = response.css('span.titular::text').get()
            extract_text = response.css('p').getall()
            
        elif response.url.startswith('https://www.lesoir.be/'):
            headline = response.css('article h1::text').get()
            extract_text = response.css('p').getall()
            lang = 'fr'
            
        elif response.url.startswith('https://www.news.com.au/'):
            headline = response.css('h1.story-headline::text').get()
            extract_text = response.css('p').getall()
            lang = 'en'

        # Realizamos y primer prepocesamiento de texto para evitar problemas a la hora de guardar el csv    
        extract_text = " ".join(extract_text)
        # Elimina todas las etiquetas
        headline = BeautifulSoup(headline, "lxml").text
        clean_text = BeautifulSoup(extract_text, "lxml").text
        # Elimina caracteres problemáticos
        for char in list_replace:
            clean_text = clean_text.replace(char, "").strip()
            headline = headline.replace(char, "").strip()
        print(f"\"{today}\",\"{response.url}\",\"{headline}\",\"{clean_text}\",\"{lang}\"", file=filep)


In [5]:
# Lanzamos el proceso de Crawler
process = CrawlerProcess({
    'USER_AGENT': 'Google SEO Bot'
})


filep = open(write_path, 'a')
process.crawl(news_spider)
process.start()
filep.close()

RROR: Spider error processing <GET https://elviajero.elpais.com/elviajero/2020/09/03/actualidad/1599132885_709554.html> (referer: None)
Traceback (most recent call last):
  File "/Users/alejandrolopezsanchez/opt/anaconda3/lib/python3.7/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/Users/alejandrolopezsanchez/opt/anaconda3/lib/python3.7/site-packages/scrapy/spiders/__init__.py", line 90, in _parse
    return self.parse(response, **kwargs)
  File "<ipython-input-4-4cb260b1ecdf>", line 63, in parse
    headline = BeautifulSoup(headline, "lxml").text
UnboundLocalError: local variable 'headline' referenced before assignment
2020-09-10 18:35:15 [scrapy.core.scraper] ERROR: Spider error processing <GET https://plus.lesoir.be/324014/article/2020-09-09/visa-pour-la-flandre-sanda-dia-et-chovanec-secouent-les-consciences-flamandes> (referer: None)
Traceback (most recent call last):
  File "/Users/alejandrol

In [7]:
df_news = pd.read_csv(write_path)
df_news.head(10)

,scraping_date,url,headline,text,lang
0,2020-09-10,https://www.20minutos.es/noticia/4375204/0/dav...,David Escors: Estamos yendo más rápido de lo q...,A raíz de la suspensión de los ensayos clínic...,es
1,2020-09-10,https://www.20minutos.es/noticia/4375385/0/un-...,Un vídeo muestra lo lejos que puede llegar el ...,Que el coronavirus se expande de forma rápid...,es
2,2020-09-10,https://www.20minutos.es/noticia/4373963/0/ast...,Asturias el 'oasis' en una España barrida por ...,Asturias se ha convertido también en esta segu...,es
3,2020-09-10,https://www.20minutos.es/noticia/4375219/0/la-...,La Policía dice que Interior pagó 53.000 euros...,El informe de la Policía Nacional que ha rec...,es
4,2020-09-10,https://www.20minutos.es/noticia/4375386/0/dir...,Casado: La votación es un triunfo del municipa...,El Pleno del Congreso ha rechazado este jueves...,es
5,2020-09-10,https://www.20minutos.es/noticia/4362007/0/los...,Las servilletas de bar la otra víctima que el ...,Las medidas antiCovid del Gobierno central ...,es
6,2020-09-10,https://www.20minutos.es/noticia/4375035/0/res...,El camino de la Covid desemboca otra vez en la...,Fueron los más castigados por la primera ola d...,es
7,2020-09-10,https://www.20minutos.es/noticia/4375123/0/edu...,Educación deja sin plaza a docentes por cumpli...,La Confederación General del Trabajo (CGT) a...,es
8,2020-09-10,https://www.20minutos.es/noticia/4375388/0/oli...,Olivia Molina: Si hubiera una valla yo estaría...,En la serie La valla de Antena 3 (que se estre...,es
9,2020-09-10,https://www.20minutos.es/noticia/4375384/0/cas...,Así es el casoplón que Cristina Pedroche y Dab...,Cristina Pedroche y Dabiz Muñoz han dejado s...,es
